1 Les données

1.1 Chargement des données JSON

Chaque enregistrement contenu dans le fichier regroupements_arbres.json représente un site
de la ville contenant un ou plusieurs arbres. Ce site peut regrouper un ou plusieurs arbres d’un
même genre ou bien des arbres de variétés multiples. Par abus de langages, dans la suite, le
terme arbre désignera un site, c’est à dire une entrée de ce jeu de données. On choisit ici de
représenter chaque arbre par un dictionnaire Python.

In [1]:
#Q1 : une fonction permettant de charger un fichier JSON et de le convertir en liste de dictionnaires Python.
import matplotlib
import folium
import json
import heapq
def json_to_list(file_path):
    with open(file_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)
        if isinstance(data, list):
            return data
        else:
            raise ValueError("Le fichier JSON ne contient pas une liste au niveau racine.")
liste_tp = json_to_list("regroupements_arbres.json")
print(liste_tp[0].keys())

dict_keys(['geo_point_2d', 'geo_shape', 'gml_id', 'id_regroupement', 'denomination', 'date_plantation', 'nb_crea', 'nb_actu', 'note_previ_renouv', 'date_dern_taille', 'frequence_taille', 'date_future_taille', 'nuquart', 'numelag', 'gestionnaire', 'type_taille', 'genre', 'espece', 'variete'])


In [2]:
#Q2 une fonction de manière à re-structurer votre liste de dictionnaires (représen-
#tant chacun un arbre) en ajoutant un identifiant unique et en ne conservant que les données utiles
def restructuration_liste_de_dicos(liste):
    compteur = 0
    cles_a_supprimer = ['geo_shape', 'gml_id', 'id_regroupement', 'nb_crea', 'nb_actu', 'note_previ_renouv', 'date_dern_taille', 'frequence_taille', 'date_future_taille', 'nuquart', 'numelag', 'gestionnaire', 'type_taille', 'espece', 'variete']
    for dico in liste:
        for cle in cles_a_supprimer:
            del dico[cle]
        dico["id"] = compteur
        compteur += 1
restructuration_liste_de_dicos(liste_tp)
print(liste_tp[0].keys())

dict_keys(['geo_point_2d', 'denomination', 'date_plantation', 'genre', 'id'])


In [3]:
#Q3 Affichez les 5 premiers
print(liste_tp[:5])

[{'geo_point_2d': {'lon': -1.6400023355317408, 'lat': 48.09284087998131}, 'denomination': 'Place du Ronceray', 'date_plantation': 2001, 'genre': 'Multiple', 'id': 0}, {'geo_point_2d': {'lon': -1.6441084416780685, 'lat': 48.091173704447286}, 'denomination': 'Boulevard Paul Hutin-Desgrées', 'date_plantation': 2001, 'genre': 'Multiple', 'id': 1}, {'geo_point_2d': {'lon': -1.6578238933001286, 'lat': 48.09083480664933}, 'denomination': 'Ecole Maternelle Henri Wallon', 'date_plantation': None, 'genre': 'Multiple', 'id': 2}, {'geo_point_2d': {'lon': -1.6823902887277506, 'lat': 48.09860215750543}, 'denomination': 'GS Villeneuve', 'date_plantation': None, 'genre': 'Tilia', 'id': 3}, {'geo_point_2d': {'lon': -1.6861685364818846, 'lat': 48.099621528774755}, 'denomination': 'Rue Ginguené', 'date_plantation': 2000, 'genre': 'Acer', 'id': 4}]


1.2 Visualisation des arbres sur une carte

In [4]:
#Q4+5 une carte centrée sur Rennes et les marqueurs affichent des informations comme le genre de
#l’arbre et sa date de plantation dans une info-bulle
tiles="recensement d'arbres dans la ville de Rennes"
Carte = folium.Map(location = [48.1173, -1.6778],zoom_start = 13)
for d in liste_tp :
    location=[d['geo_point_2d']['lat'],d['geo_point_2d']['lon']]
    text='genre= '+ d['genre'] + 'date de plantation= '+str(d['date_plantation'])
    marqueur = folium.Marker(location = location,popup = text)
    marqueur.add_to(Carte)
display(Carte)

In [5]:
#Q6 Enregistrez votre carte dans un fichier html
Carte.save("recensement d’arbres dans la ville de Rennes.html")

1.3 Calcul des distances entre les arbres

In [6]:
#Q7 Implémentation de la formule de Haversine pour calculer la distance en mètres entre
#deux points géographiques connaissant leur latitude et leur longitude (sur Terre).
from math import *
def distance(arbre1,arbre2):
    lat1=arbre1["geo_point_2d"]['lat']
    lon1=arbre1["geo_point_2d"]['lon']
    lat2=arbre2["geo_point_2d"]['lat']
    lon2=arbre2["geo_point_2d"]['lon']
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # rayon de la terre.
    return c * r * 1000
print(distance(liste_tp[0],liste_tp[1]))

356.8893969976452


In [7]:
#Q8  les paires d’arbres les plus proches ? Les plus éloignés ?
m=100
M=0
m_liste=[0,0]
M_liste=[0,0]
for i in range(len(liste_tp)-1):
    for j in range(i+1,len(liste_tp)):
        if m > distance(liste_tp[i],liste_tp[j]):
            m=distance(liste_tp[i],liste_tp[j])
            m_liste=[i,j]
        if M <distance(liste_tp[i],liste_tp[j]):
            M=distance(liste_tp[i],liste_tp[j])
            M_liste=[i,j]
print('distance minimale est: '+str(m)+"elle est entre l'arbre"+str(m_liste[0])+"et l'arbre"+str(m_liste[1]))
print('distance maximale est: '+str(M)+"elle est entre l'arbre"+str(M_liste[0])+"et l'arbre"+str(M_liste[1]))

distance minimale est: 0.02645370541952768elle est entre l'arbre190et l'arbre1357
distance maximale est: 9469.461747629788elle est entre l'arbre667et l'arbre1128


1.4 Création d’un graphe
Nous allons maintenant créer un graphe d’arbres. Dans ce graphe non orienté, les noeuds
représentent les arbres et les distances entre les arbres étiquettent les arêtes.

Q9)Une structure de données permettant de représenter un tel graphe: ce que je vais appeler une pseudo liste d'adjacence.

d={i:[les distances entre le sommet i et les sommet j (for j in range(n))]for i in range(n)}

n=nombre des sommet 
le dictionnaire contien des listes qui contiennent les distances entre chaque sommet et tous les autres sommets
utiliser des liste d'adjacence est inutil. Il ne sert a rien de préciser quels sont les sommets lié a chaque sommet i vu que les sommets sont tous liée

In [8]:
#Q)10 une fonction permettant de créer le graphe complet 
#(dans un graphe complet, chaque noeud est relié à tous les autres).
def liste_dadjacence(liste_tp):
    d={}
    n=len(liste_tp)
    for i in range(n):
        d[i]=[]
        for j in range(n):
            if not j==i:
                d[i].append((j,distance(liste_tp[i],liste_tp[j])))
    return d
liste_dadjacence=liste_dadjacence(liste_tp)
print(liste_dadjacence[1][:3])

[(0, 356.8893969976452), (2, 1019.3788224551483), (3, 2960.637282114872)]


In [9]:
#Q)11 un test qui permette de vérifier que le graphe généré à l’aide de la 
#fonction précédente est effectivement complet et contient le nombre attendu de noeuds et d’arrêtes.
def test_graph_de_données(liste_tp,liste_dadjacence):
    n=len(liste_tp)
    test1=len(liste_dadjacence.keys())==n # True si il y a les n sommets dans la structure de données
    test2=True
    for i in range(n):
        if not(len(liste_dadjacence[i])==n-1):#True si il y a les n-1 arrêtes(sans l'arret qui est
            teste2=False                      #de distance nulle etre l'arbre i et lui même (d'ou viens le n-1)) 
    return (test1 and test2)                  #pour chaque sommets dans la structure de données
print(test_graph_de_données(liste_tp,liste_dadjacence))

True


2 Butinage

Selon [1], Les vols d’orientation réalisés par les abeilles débutantes, juste après avoir quitté la
ruche, sont limités à de courtes distances (typiquement quelques centaines de mètres). Ces vols
permettent aux jeunes ouvrières de se familiariser avec l’environnement.

Nous supposons dans la suite que les abeilles juvéniles volent d’arbre en arbre.
On suppose aussi qu’elles ne peuvent se reposer que sur un arbre identifié dans
le jeu de données (les fleurs sur les rebords des fenêtres ne comptent pas, les
voitures sont trop dangereuses et les abribus trop fréquentés..)

Supposons, pour l’instant, qu’une abeille juvénile ne puisse pas parcourir plus de 200m
sans devoir faire de pause sur un arbre. Nous ne tenons pas compte ici des obstacles liés à
l’environnement.

In [10]:
"""Une fonction qui extrait le graphe graphe_200 sur le modèle de celui dela section précédente.
Dans ce graphe, les noeuds sont les arbres, deux arbres sont reliés par une arrête lorsque la distance entre  
ces arbres est inférieure ou égale à 200m. Les arrête sont étiquetées par la distance entre ces arbres."""
def graphe_200(liste_dadjacence):
    d={}
    n=len(liste_dadjacence.keys())
    for i in range(n):
        d[i]=[]
        for j in range(n-1):
            if liste_dadjacence[i][j][1]<=200:
                d[i].append(liste_dadjacence[i][j])
    return d
graphe_200=graphe_200(liste_dadjacence)
print(graphe_200[0])

[(220, 116.67161984055961), (233, 192.0526116973141), (238, 173.02172800255823), (607, 156.66469835504614), (811, 102.48907672344053), (814, 148.2990174090289), (820, 65.46674955611417), (821, 74.0291725735591), (848, 183.95917596352882), (850, 79.41903620089553), (1103, 188.9850734419168), (1139, 104.73970670669219), (1160, 122.64431426053547), (1161, 150.56647574817634), (1440, 148.14318997144784)]


In [11]:
#Q13 le nombre d’arêtes compte graphe_200
def nombre_arrête(graph):
    s=0
    for i in graph.values():
        s+=len(i)
    return s/2
nombre_arrête=nombre_arrête(graphe_200)
print(nombre_arrête)

6386.0


In [12]:
#Q14) le nombre maximal de voisins qu’un arbre puisse avoir
def num_max_voisins(graph):
    m=0
    for i in graph.keys():
        if len(graph[i])>m:
            m=len(graph[i])
    return(m)
m = num_max_voisins(graphe_200)
print('nombre maximale de voisins est '+str(m))

nombre maximale de voisins est 29


In [13]:
#Q15) les arbres (dénomination, genre et date de plantation) ayant un nombre de voisins maximal
def arbres_max_voisins(graph,liste_tp,m):
    arbres_ids=[]
    for i in graph.keys():
        if len(graph[i])==m:
            arbres_ids.append(i)
    return [(liste_tp[i]['id'],liste_tp[i]['denomination'],liste_tp[i]['genre'],liste_tp[i]['date_plantation']) for i in arbres_ids]
arbres_max_voisins=arbres_max_voisins(graphe_200,liste_tp,m)
print(arbres_max_voisins)

[(795, 'Rue Jules Andrade', 'Non renseigné', 2022), (1047, 'Parking Square de Guyenne', 'Platanus', 1971)]


In [14]:
#Q16) Representation sur une carte de la ville les arbres (et leur voisins) identifiés
tiles="arbres ayant un nombre maximal de voisins dans graphe_200"
Carte = folium.Map(location = [48.1173, -1.6778],zoom_start = 13)
for tupl in arbres_max_voisins : #tupl[0]==id de l'arbre identifié
    #affichage des deux arbres identifiés
    location=[liste_tp[tupl[0]]['geo_point_2d']['lat'],liste_tp[tupl[0]]['geo_point_2d']['lon']] 
    text='genre= '+ liste_tp[tupl[0]]['genre'] + ' date de plantation= '+str(liste_tp[tupl[0]]['date_plantation'])
    marqueur = folium.Marker(location = location,popup = text)
    marqueur.add_to(Carte)
    #affichage de leurs voisins
    for couple in graphe_200[tupl[0]]:
        location=[liste_tp[couple[0]]['geo_point_2d']['lat'],liste_tp[couple[0]]['geo_point_2d']['lon']] 
        text='genre= '+ liste_tp[couple[0]]['genre'] + ' date de plantation= '+str(liste_tp[couple[0]]['date_plantation'])
        marqueur = folium.Marker(location = location,popup = text)
        marqueur.add_to(Carte)
display(Carte)
Carte.save("arbres ayant un nombre maximal de voisins dans graphe_200.html")

In [15]:
#17)  sites d’arbres joignables partant de arbre_a (resp. arbre_g) lors de l’exploration d’une abeille juvénile
#on écrit essentiellement un algorithme de parcours de graphe en profondeur
def nb_sites_joignables(G, arbre_id, deja_visite):
    if arbre_id not in deja_visite:
        count=1   # Compter le noeud actuel
        deja_visite.append(arbre_id)

        #Parcourir les voisins du noeud actuel
        for neighbor in G[arbre_id]:
            count+= nb_sites_joignables(G,neighbor[0],deja_visite)
        return count
    return 0
print(nb_sites_joignables(graphe_200, 795, []))
print(nb_sites_joignables(graphe_200, 1047, []))
"""def nb_site_joignables(G,arbre_id):
    deja_visite=[arbre_id]
    file=[arbre_id]
    while not file==[]:
        for neighbor in G[file[0]]:
            if neighbor[0] not in deja_visite:
                deja_visite.append(neighbor[0])
                file.append(neighbor[0])
        file.pop(0)
    return (len(deja_visite))
print(nb_sites_joignables(graphe_200, 795))
print(nb_sites_joignables(graphe_200, 1047))"""

36
199


'def nb_site_joignables(G,arbre_id):\n    deja_visite=[arbre_id]\n    file=[arbre_id]\n    while not file==[]:\n        for neighbor in G[file[0]]:\n            if neighbor[0] not in deja_visite:\n                deja_visite.append(neighbor[0])\n                file.append(neighbor[0])\n        file.pop(0)\n    return (len(deja_visite))\nprint(nb_sites_joignables(graphe_200, 795))\nprint(nb_sites_joignables(graphe_200, 1047))'

Q18)Figure 2: Arbres ayant un nombre maximal de voisins dans graphe_200
Question 18: Pourquoi le nombre de sites d’arbres joignable à partir de arbre_a est-il différent
du nombre de sites d’arbres joignables à partir de arbre_g ? 
le graphe n'est pas connexe: il y aura des reptures ( plusieurs composantes connexes une ou il y a l'arbre a et l'autre arbre g) 

In [16]:
#Q19: distance
def graphe_200(liste_dadjacence,D):
    d={}
    n=len(liste_dadjacence.keys())
    for i in range(n):
        d[i]=[]
        for j in range(n-1):
            if liste_dadjacence[i][j][1]<D and i != j:
                d[i].append(liste_dadjacence[i][j])
    return d
def distance_minimale_99(liste_dadjacence):
    proportion = 0
    nb_total_arbres=len(liste_dadjacence)
    dist = 352
    while proportion < 0.99:
        proportion=nb_sites_joignables(graphe_200(liste_dadjacence,dist), 1047, [])/nb_total_arbres
        print(dist, proportion)
        dist += 0.1
    return dist -0.1
distance_minimale_99=distance_minimale_99(liste_dadjacence)
print("Donc au final distance_minimale_99 ="+str(distance_minimale_99)+"m")


352 0.9627118644067797
352.1 0.9627118644067797
352.20000000000005 0.9911864406779661
Donc au final distance_minimale_99 =352.20000000000005m


In [17]:
#Q20 
# obtention du plus grand sous-graphe connexe de graphe_355_nc
def largest_connected_component(graph):
    """
    Trouve la plus grande composante connexe d'un graphe.

    :param graph: dict, un dictionnaire représentant le graphe sous la forme {sommet: [(voisin, poids), ...]}
    :return: set, un ensemble contenant les sommets de la plus grande composante connexe
    """
    def dfs(node, visited):
        """Effectue une recherche en profondeur (DFS) et retourne les sommets de la composante connexe."""
        stack = [node]
        component = set()
        while stack:
            current = stack.pop()
            if current not in visited:
                visited.add(current)
                component.add(current)
                stack.extend(neighbor for neighbor, _ in graph[current])
        return component

    visited = set()
    largest_component_nodes = set()

    for node in graph:
        if node not in visited:
            component = dfs(node, visited)
            if len(component) > len(largest_component_nodes):
                largest_component_nodes = component

    # Construire le sous-graphe correspondant à la plus grande composante connexe
    largest_component_graph = {
        node: [(neighbor, weight) for neighbor, weight in graph[node] if neighbor in largest_component_nodes]
        for node in largest_component_nodes
    }

    return largest_component_graph
def graphe_355_nc(liste_tp):
    graphe_355_nc = {}
    for arbre in liste_tp:
        graphe_355_nc[arbre['id']] = []
        for arbre_voisin in liste_tp:
            dist = distance(arbre, arbre_voisin)
            if arbre_voisin == arbre or dist>355:
                continue
            graphe_355_nc[arbre['id']].append((arbre_voisin['id'], dist))
    return graphe_355_nc
graphe_355_nc=graphe_355_nc(liste_tp)
graphe_355=largest_connected_component(graphe_355_nc)
print(sum([len(a) for a in graphe_355.values()])/2)

16845.0


q21 :
j'aiplacé une ruche près d’un platane du "Parking Square de Guyenne". les abeilles
sont encore jeunes et doivent optimiser leur déplacement. les abeilles recherchent en priorité
les arbres mellifères c’est à dire des arbres qui produisent du nectar ou du pollen en quantité
suffisante pour être utiles aux abeilles. Parmi les genres d’arbres répertoriés dans votre jeu de
données, les genres suivants sont considérés comme mellifères:
1 arbres_melliferes = {’Acer’, ’Alnus’, ’Betula’, ’Castanea’, ’Corylus’, ’Crataegus’, ’Gleditsia’
, ’Morus’, ’Pyrus’, ’Robinia’, ’Salix’, ’Sorbus’, ’Sophora’, ’Tilia’, ’Ulmus’}
Les arbres du genre Salix (Saule) sont optimaux pour les abeilles juvéniles. En effet, les
saules fleurissent très tôt au printemps, le pollen des saules est riche en protéines, les fleurs des
saules sont simples et facilement accessibles, même pour une abeille inexpérimentée.
 on cherche l’arbre le plus intéressant pour une abeille juvénile en supposant
que: 
Nos abeilles n’empruntent que les plus courts chemins1.
• Plus un arbre est vieux plus il porte des fleurs et donc plus il est intéressant. On suppose
les arbres de destinations ont un intérêt égal à 4 fois leur âge.
• Plus un chemin est long moins il est intéressant. On suppose que chaque dizaine de mètres
parcourue coute -1
• Les arbres mellifères sur les chemins compensent un peu les distances parcourues. On
suppose que chaque arbre mellifère apporte un bonus de 1.

In [18]:
"""On cherche le meilleur arbre. Pour cela on dispose d'un moyen de classer les arbres, avec un score : 
score(arbre_destination) : 4*age_de_l'arbre_dest + nombre_d'arbres_mellifères_sur_le_chemin - longueur_du_chemin_en_mètres/10

De plus, on part d'une ruche au parking square de Guyenne (id : 1047), et on n'emprunte que les plus courts chemins. 

Il faut stocker dans une liste/dictionnaire les scores des arbres associés aux id des arbres. 
Pour obtenir le score de chaque arbre, on utilise d'abord un algorithme de Dijkstra pour trouver les plus courts chemins depuis le sommet source vers tous les autres sommets du graphe. Ensuite, pour chaque sommet du graphe, on retrace le chemin (plus court chemin bien sûr) entre ce sommet et le sommet source, en comptant le nombre d'arbres mellifères traversés. On ajoutera 4 * l'âge de l'arbre destination, si celui-ci est mellifère, pour obtenir le score complet. 

Enfin, on cherche dans la liste/dictionnaire obtenu(e) l'arbre ayant le plus haut score."""

# confirmation de l'id de l'arbre source : 
# print([arbre for arbre in liste_tp if arbre['denomination']=='Parking Square de Guyenne'])
# on obtient bien un id de 1047


# algorithme de plus court chemin
def dijkstra(graph, start):
    """
    Trouve les plus courts chemins depuis le sommet `start` vers tous les autres sommets dans un graphe.

    :param graph: dict, un dictionnaire représentant le graphe sous la forme {sommet: [(voisin, poids), ...]}
    :param start: sommet de départ
    :return: tuple (distances, parents)
        - distances: dict avec les distances minimales depuis `start` vers chaque sommet
        - parents: dict indiquant le parent de chaque sommet dans le chemin le plus court
    """
    # Initialisation des distances et de la file de priorité
    distances = {node: float('inf') for node in graph}  # Distances infinies
    distances[start] = 0
    parents = {node: None for node in graph}           # Aucun parent initialement

    priority_queue = [(0, start)]  # File de priorité (distance, sommet)

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        # Si la distance actuelle est plus grande que la meilleure distance connue, ignorer
        if current_distance > distances[current_node]:
            continue

        # Exploration des voisins
        for neighbor, weight in graph[current_node]:
            distance = current_distance + weight

            # Si un chemin plus court vers le voisin est trouvé
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                parents[neighbor] = current_node
                heapq.heappush(priority_queue, (distance, neighbor))

    return distances, parents
arbres_melliferes = ['Acer', 'Alnus', 'Betula', 'Castanea', 'Corylus', 'Crataegus', 'Gleditsia', 'Morus', 'Pyrus', 'Robinia', 'Salix', 'Sorbus', 'Sophora', 'Tilia', 'Ulmus']
def meilleur_saule_id(graphe_355,arbres_melliferes):
    dist, parents = dijkstra(graphe_355, 1047)
    arbres_melliferes = ['Acer', 'Alnus', 'Betula', 'Castanea', 'Corylus', 'Crataegus', 'Gleditsia', 'Morus', 'Pyrus', 'Robinia', 'Salix', 'Sorbus', 'Sophora', 'Tilia', 'Ulmus']

    # liste des scores des arbres dest, c'est une liste de tuples [ (arbre dest, score), ...]
    scores = []

    # obtention des scores pour tous les arbres de graphe_355:
    for arbre_dest in graphe_355.keys(): # l'arbre destination est représenté par son id

        score = dist[arbre_dest]/10 # prise en compte de la longueur du chemin le plus court vers l'arbre de destination

        # initialisation d'un arbre intermediaire, qui part du sommet dest et remonte jusqu'au sommet source. 
        # cet arbre intermediaire servira à compter le nombre d'arbres mellifères sur le chemin
        arbre_intermediaire = arbre_dest 

        # boucle qui compte le nombre d'arbres mellifères sur le chemin et actalise le score en fonction
        while arbre_intermediaire != 1047:
            type = liste_tp[arbre_intermediaire]['genre'] # obtient le type de l'arbre intermédiaire
            if type in arbres_melliferes: # ajoute 1 au score si l'arbre intermédiaire est mellifère
                score += 1

            arbre_intermediaire = parents[arbre_intermediaire] # on remonte vers le sommet source

        # ajoute quatre fois l'âge de l'arbre destination si celui-ci est mellifère
        carac = (liste_tp[1047]['genre'], liste_tp[1047]['date_plantation'])
        if carac[0] in arbres_melliferes and carac[1] is not None:
            score += 4 * (2024-carac[1])

        scores.append( (arbre_dest, score) )
    """On a les scores de tous les arbres, ce qui nous permet de trouver l'arbre ayant le plus haut score.
    Cependant, on ne cherche pas exactement l'arbre ayant le plus haut score, mais plutôt le saule ayant le plus haut score.
    On regarde quand même le meilleur arbre : si c'est un saule alors ce sera aussi le meilleur saule - question terminée."""
    # on a ici les scores de tous les arbres. on peut donc trouver l'arbre ayant le plus haut score
    meilleur_arbre = max(scores, key = lambda x: x[1])
    print("l'id du meilleur arbre est : ", meilleur_arbre[0])
    print("le genre de cet arbre est : ", liste_tp[meilleur_arbre[0]]['genre'])
    # on trouve que le meilleur arbre est de genre Multiple, on ne sait pas s'il y a des saules dedans

    # recherche du meilleur saule:

    # on trie la liste scores pour avoir les arbres par score décroissant
    scores_sorted = sorted(scores, key=lambda x: x[1], reverse = True)
    # on parcours la liste triée jusqu'a trouver un saule:
    for i in scores_sorted:
        genre = liste_tp[i[0]]['genre']
        if genre == 'Salix':
            meilleur_saule = i
            break
    return meilleur_saule[0]
meilleur_saule_id=meilleur_saule_id(graphe_355,arbres_melliferes)
print("l'id du meilleur saule est : ", meilleur_saule_id)
print(liste_tp[meilleur_saule_id])


l'id du meilleur arbre est :  786
le genre de cet arbre est :  Multiple
l'id du meilleur saule est :  1398
{'geo_point_2d': {'lon': -1.648891779123316, 'lat': 48.10949763784054}, 'denomination': 'Rue Jules Andrade', 'date_plantation': 2022, 'genre': 'Salix', 'id': 1398}


Q22) D'après notre cours, on sait que l'algorithme de Dijksra est le plus rapide lorsque les poids des arrêtes sont tous positifs.

4 Placement de ruches


Nous nous intéressons maintenant au pollen que peuvent récolter des abeilles durant une journée.
Nous supposons ici que
• Seuls les arbres mellifères produisent du pollen.
• Chaque arbre mellifère produit en moyenne 100g de pollen par jour
• Une abeille butineuse typique récolte environ 10mg de pollen sur un arbre mellifère
• Lorsqu’une abeille a récolté 10mg de pollen sur un arbre mellifère, elle doit retourner à la
ruche
• Une abeille ne s’éloigne pas à plus de 2km de sa ruche
• Les ruches sont placés à coté des arbres
Nous considérons toujours que les abeilles peuvent voler jusqu’à 355m avant de devoir
se reposer sur un arbre. (Autrement dit, vous pouvez continuer à travailler dans le graphe
graph_355)

In [19]:
#Q23 :une fonction qui calcule les arbres mellifères atteignables par une abeille partant d’un arbre donné dans graph_355.
#à partir de l'arbre source, on fait un algo de Dijkstra pour avoir les plus courts chemins vers tous les autres 
# arbres de graphe_355. l'algo renvoie la liste "distances" des distances minimales à partir du sommet source. 
# avec cela on sélectionne les arbres à moins de 2km du sommet source, puis on séléctionne parmi eux les arbres mellifères.
def arbres_melliferes_atteignables(arbre_source):
    distances, parents = dijkstra(graphe_355, arbre_source)
    l = []
    for i in graphe_355.keys():
        if distances[i] <= 2000 and liste_tp[i]['genre'] in arbres_melliferes:
            l.append(i)
    return l

La quantité de pollen accessible par les abeilles d’une ruche située sur le platane situé Parking Square de Guyenne ?

Q25 : on a la liste des arbres mellifères atteignables à partir du parking square de Guyenne avec la fonction de la question 23. chaquun de ces arbers produit 100g de pollen par joura

In [20]:
#Q 25 : on a la liste des arbres mellifères atteignables à partir du parking square de Guyenne avec la fonction de 
#Q23. chaquun de ces arbers produit 100g de pollen par jour
def quantite_pollen_par_jour_parking_guyenne(arbres_melliferes_atteignables): 
    return(len(arbres_melliferes_atteignables(1047))*100) # résultat en grammes
quantite_pollen_par_jour_parking_guyenne=quantite_pollen_par_jour_parking_guyenne(arbres_melliferes_atteignables)
print(str(quantite_pollen_par_jour_parking_guyenne) + " grammes/jour")

6700 grammes/jour


Q26 Hypothèses: une abeille peut transporter 10mg de pollen par voyage, et qu’une abeille butineuse (courageuse) peut réaliser deux voyages par jour. 

Le nombre abeilles butineuses dans votre ruche pour pour collecter tout le pollen produit par les arbres mellifères atteignables partant du platane situé Parking Square de Guyenne en une journée :

In [21]:
#capacité d'une abeille = 20 mg/jour = 0.02g/jour ;
#quantité totale de pollen = 6700 g/jour ;
#nombre d'abeilles = quantité totale de pollen / capacité d'une abeille
nombre_abeilles = quantite_pollen_par_jour_parking_guyenne/0.02
print(nombre_abeilles)

335000.0


Q27) Un de mes amis est apiculteur et souhaite installer des ruches au pied des arbres dans la métropole. je dpos lui proposer une répartition de moins de 50 ruches telle que chaque arbre mellifère est accessible par les abeilles d’au moins une ruche

In [22]:
#Principe de l'algorithme : on part d'un arbre de graphe_355 : on mettra une ruche ici. On regarde tous les arbres atteignables à partir de 
# celui-ci : on ne mettra pas de ruche à côté de ces arbres. On prend un des arbres restants. on répète le processus en 
# regardant les arbres accessibles à partir de celui-ci. 
#On fait cela de manière récursive. 
#On s'arrete quand on a fait 50 fois le processus (ce qui correspond à poser 50 ruches) ou lorsque tous les arbres
#mellifères sont atteignables.
l_355 = graphe_355.keys()

l_m = [] # liste de tous les arbres mellifères de graph_355
for i in l_355:
    if liste_tp[i]['genre'] in arbres_melliferes:
        l_m.append(i)


def arbres_atteignables(arbre_source):
    distances, parents = dijkstra(graphe_355, arbre_source)
    l = []
    for i in graphe_355.keys():
        if distances[i] <= 2000:
            l.append(i)
    return l



def f(liste_choisis):
    
    l_atteignables = [] # liste des arbres atteignables par au moins une abeille
    l_m_atteignables = [] # liste des arbres mellifères atteignables par au moins une abeille

    for i in liste_choisis:
        l_m_atteignables = l_m_atteignables + arbres_melliferes_atteignables(i)
        l_atteignables = l_atteignables + arbres_atteignables(i)
    
    c = True
    for i in l_m:
        if i not in l_m_atteignables:
            c = False
    
    
    if c: # c est vrai lorsque tous les arbres mellifères sont atteignables à partir des ruches déjà posées
        print("tous les arbres mellifères sont atteignables, " + str(liste_choisis[0]))
        return liste_choisis


    if len(liste_choisis)==50: # si 50 ruches ont déjà été posées, on n'en pose pas plus, on retourne les 50 ruches 
        return liste_choisis 


    for i in l_355:
        if i not in l_atteignables:
            liste_choisis.append(i)
            break
    
    return f(liste_choisis)

l_ruches = f([])
print(l_ruches)
print(f([50]))
print(f([51]))
print(f([1047]))

"""
l_ruches_tout = [f([i]) for i in l_355]
l_ruches_tout.sort(key=len)
print(l_ruches_tout[0])
"""

l_ruches = f([307]) # en commançant à 307 on a une distribution de seulement 9 ruches
print(l_ruches)



tous les arbres mellifères sont atteignables, 0
[0, 3, 8, 20, 32, 45, 49, 57, 107, 423, 471, 797]
tous les arbres mellifères sont atteignables, 50
[50, 0, 3, 20, 32, 33, 45, 49, 57, 107, 471, 797]
tous les arbres mellifères sont atteignables, 51
[51, 0, 5, 14, 20, 33, 45, 49, 57, 797]
tous les arbres mellifères sont atteignables, 1047
[1047, 0, 3, 20, 32, 45, 49, 57, 107, 138, 471, 797]
tous les arbres mellifères sont atteignables, 307
[307, 2, 3, 8, 20, 49, 57, 423, 471]


discussion de la complexité du problème puis de l’efficacité et de l’optimalité de ma solution:
La solution trouvée est très peu optimale, mais trouver la solution optimale par force brute est très long.
Néanmoins on peut obtenir une solution un peu optimisée avec le code commenté en bas de la rubrique de code précédente.
Cette rubrique est commentée car son exécution est très lente, mais elle donne une répartition avec seulement ___ ruches.

In [23]:
#Q28)  proposition de placement sur une carte.
latitude = 48.117266
longitude = -1.6777926
carte_rennes_bis_bis = folium.Map(location=[latitude, longitude], zoom_start=13)

arbres = [liste_tp[i] for i in l_ruches]

# Ajout des marqueurs
for arbre in arbres:
    folium.Marker(
        location = [ arbre['geo_point_2d']['lat'] , arbre['geo_point_2d']['lon']],
        popup = str(arbre['denomination']) +
            "; date plantation : " + str(arbre['date_plantation']) + 
            "; genre : " + str(arbre['genre'])
    ).add_to(carte_rennes_bis_bis)

# Sauvegarde de la carte dans un fichier HTML
carte_rennes_bis_bis.save("carte_rennes_bis_bis.html")

display(carte_rennes_bis_bis)